<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2025notebooks/2025_1003kalman_filter_olivetti_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config InlineBackend.figure_format = 'retina'
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

import numpy as np
# numpy の表示桁設定
np.set_printoptions(precision=5, suppress=False)

# オリベッティ顔データセットの読み込み
from sklearn.datasets import fetch_olivetti_faces
data = fetch_olivetti_faces()
X, y = data.data, data.target
print(f'訓練データサイズ X.shape:{X.shape}')
print(f'教師データサイズ y.shape:{y.shape}')
print(data.DESCR)

In [ ]:
# データの表示
nrows = 40   # nrows 人分のデータを表示
ncols = 10
fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 1.4, nrows * 1.4), constrained_layout=True)
# constrained_layout は subplot や 凡例やカラーバーなどの装飾を自動的に調整して，
# ユーザが要求する論理的なレイアウトをできるだけ維持しながら， 図ウィンドウに収まるようにします。

for i in range(nrows):
    for j in range(ncols):
        x = i * 10 + j
        fig_axes[i][j].imshow(X[x].reshape(64,64), cmap='gray')
        fig_axes[i][j].axis('off')
        fig_axes[i][j].set_title(f'num:{x}, ID:{y[x]}')

plt.show()


データであるオリベッティ顔画像データセットは 400 枚の画像はそれぞれ縦横 64 画素である。これを 1 次元にすれば 1 枚の画像は 4096 次元のベクトルである。
したがって，系列予測課題とみなすこともできる。


In [ ]:
# 表示させたい画像番号を指定するリスト
image_nums = [0, 10, 20, 30]

figs, axes = plt.subplots(len(image_nums), 2, figsize=(12, 1.5 * len(image_nums)), tight_layout=False)
for i, image_num in enumerate(image_nums):
    x = X[image_num]

    #figs, axes = plt.subplots(1,2, figsize=(12,3), tight_layout=False)
    #figs.suptitle(f'データ番号:{N}')

    axes[i][0].plot(x)
    axes[i][0].set_ylim(0,1)

    axes[i][1].imshow(x.reshape(64,64), cmap='gray')
    axes[i][1].set_xticks([])
    axes[i][1].set_yticks([])

plt.show()

In [ ]:
try:
    import filterpy
except ImportError:
    !pip install filterpy
    import filterpy

from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise


In [ ]:
N=70
N=90
x = X[N]

kf = KalmanFilter(dim_x=1,dim_z=1)

kf.x = np.array(x.mean())
kf.F = np.array(x.var())
kf.H = np.array(x.var())

# kf.x = np.array(0)
# kf.F = np.array(1.)
# kf.H = np.array(1.)

zs = x
xs, Cov = [], []
for z in zs:
    kf.predict()
    kf.update(z)
    xs.append(kf.x)
    Cov.append(kf.P)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(5,2))
axes[0].imshow(np.array(xs).reshape(64,64), cmap='gray')
axes[0].set_title('after filtering')
axes[1].imshow(x.reshape(64,64), cmap='gray')
axes[1].set_title('Original')
